In [1]:
import numpy as np
import pandas as pd
from pandas_datareader import data as pdata
import matplotlib.pyplot as plt

import fix_yahoo_finance as yf
yf.pdr_override() # <== that's all it takes :-)


import time
from datetime import datetime
timestamp = int(time.mktime(datetime.now().timetuple()))



#def generateSignal(pairs):

symbol = sorted(["XLF","AMG","AFL","ALL","AXP","AIG","AMP","AON","AJG","AIZ","BAC","BK","BBT","BRK.B","BLK","HRB","BHF","COF","CBOE","SCHW","CB","CINF","C","CFG","CME","CMA","DFS","ETFC","RE","FITB","BEN","GS","HIG","HBAN","ICE","IVZ","JPM","KEY","LUK","LNC","L","MTB","MMC","MET","MCO","MS","NDAQ","NAVI","NTRS","PBCT","PNC","PFG","PGR","PRU","RJF","RF","SPGI","STT","STI","SYF","TROW","TMK","TRV","USB","UNM","WFC","WLTW","XL","ZION"])

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/fix_yahoo_finance/__init__.py:43: DeprecationWarning: 
    Auto-overriding of pandas_datareader's get_data_yahoo() is deprecated and will be removed in future versions.
    Use pdr_override() to explicitly override it.
  DeprecationWarning)


In [57]:
start_date='2017-01-01'
end_date='2018-02-17'

raw_df = pdata.get_data_yahoo(symbol, start_date, end_date)

[*********************100%***********************]  69 of 69 downloaded

In [9]:
def findCorrParis(stocks, start_date, end_date, rolling_win=120):
    try:

        s = datetime.strptime(start_date,'%Y-%m-%d')
        e = datetime.strptime(end_date,'%Y-%m-%d')
        if (e-s).days<rolling_win:
            raise ArithmeticError
    except ArithmeticError:
        print("start date, end date is less than duration")

    closePx = stocks["Close"]
    #print("\n",closePx.iloc[0])

    rho =pd.rolling_corr(closePx,rolling_win)

    
    
    return rho

rho= findCorrParis(raw_df,start_date,end_date,120)

print (type(rho))

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:14: FutureWarning: pd.rolling_corr is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=120).corr(pairwise=True,other=<DataFrame>)


<class 'pandas.core.frame.DataFrame'>


In [215]:
import math
def getRollingStockPairs(rho_mat, thd ):
    dist={}
    df = pd.DataFrame(index=rho_mat.index.get_level_values(0).unique(), columns=rho_mat.columns)
    
    for date, new_df in rho.groupby(level=0):
        corr_mat=r.loc[date]
        pairs=[]
        for i in range(0,corr_mat.shape[0]):
            for j in range(0,i):
                #print ( " correlation ",corr_mat.iat[i,j], corr_mat.index[i],corr_mat.columns[j] )
                if corr_mat.iat[i,j] >= thd:
                    
                    pairs.append([corr_mat.index[i],corr_mat.columns[j],corr_mat.iat[i,j] ])
                    try:
                        if math.isnan(df.loc[date][i]):
                            df.loc[date][i]=[]
                    except:
                        tmp=1#print (".")
                    
                    #print ("-- ",df.loc[date][i], corr_mat.index[i],corr_mat.index[j])
                    df.loc[date][i].append(corr_mat.index[j])
                    #print (">> ",df.loc[date][i])
                    #    df.loc[date][i]=df.loc[date][i].extend((corr_mat.index[j]))
                   
                    try:
                        if math.isnan(df.loc[date][j]):
                            df.loc[date][j]=[]
                    except:
                        tmp=1#print (".")
                    
                    #print ("-- ",df.loc[date][j], corr_mat.index[i],corr_mat.index[j])
                    df.loc[date][j].append(corr_mat.index[i])
                    
                        
                        #df.loc[date][j]=(corr_mat.index[i])
                    #print ( " correlation ",corr_mat.iat[i,j])
        
        if (len(pairs)>0):
            dist[date]=pairs[:]
                    
                    
    return df

#test 
r = rho[-300:-1]
dist=getRollingStockPairs(r, 0.8)

print(dist["ZION"][3])
print(dist["WFC"][3])

['AFL', 'AMP', 'AXP', 'BAC', 'BBT', 'BK', 'BLK', 'C', 'CBOE', 'CFG', 'CMA', 'CME', 'COF', 'DFS', 'ETFC', 'FITB', 'GS', 'HBAN', 'ICE', 'JPM', 'KEY', 'LNC', 'MCO', 'MS', 'MTB', 'NTRS', 'PBCT', 'PGR', 'PNC', 'PRU', 'RF', 'RJF', 'SCHW', 'SPGI', 'STI', 'STT', 'SYF', 'TMK', 'TROW', 'TRV', 'USB', 'WFC', 'XLF']
['AFL', 'AMG', 'AMP', 'AXP', 'BAC', 'BBT', 'BK', 'BLK', 'C', 'CBOE', 'CFG', 'CMA', 'CME', 'COF', 'DFS', 'ETFC', 'FITB', 'GS', 'HBAN', 'ICE', 'JPM', 'KEY', 'L', 'LNC', 'LUK', 'MCO', 'MS', 'MTB', 'NTRS', 'PBCT', 'PGR', 'PNC', 'PRU', 'RF', 'RJF', 'SCHW', 'SPGI', 'STI', 'STT', 'SYF', 'TMK', 'TROW', 'USB', 'XLF', 'ZION']


In [220]:
dist["JPM"][-1]

['AFL',
 'AMP',
 'AXP',
 'BAC',
 'BBT',
 'BK',
 'BLK',
 'C',
 'CBOE',
 'CFG',
 'CMA',
 'CME',
 'COF',
 'DFS',
 'ETFC',
 'FITB',
 'GS',
 'HBAN',
 'ICE',
 'KEY',
 'LNC',
 'MCO',
 'MS',
 'MTB',
 'NTRS',
 'PBCT',
 'PGR',
 'PNC',
 'PRU',
 'RF',
 'RJF',
 'SCHW',
 'SPGI',
 'STI',
 'STT',
 'SYF',
 'TMK',
 'TROW',
 'TRV',
 'USB',
 'WFC',
 'XLF']

In [121]:
#print (rho.index[1:4])

#print (rho[0:70])

#rho['AJG'][-70:-1]
closePx = raw_df["Close"]

r = rho[0:100]
for date, new_df in r.groupby(level=0):
    print(date)
    print (r.loc[date])

2017-01-03 00:00:00
       AFL  AIG  AIZ  AJG  ALL  AMG  AMP  AON  AXP  BAC  ...   TMK  TROW  TRV  \
AFL    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN   NaN  NaN   
AIG    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN   NaN  NaN   
AIZ    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN   NaN  NaN   
AJG    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN   NaN  NaN   
ALL    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN   NaN  NaN   
AMG    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN   NaN  NaN   
AMP    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN   NaN  NaN   
AON    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN   NaN  NaN   
AXP    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN   NaN  NaN   
BAC    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN   NaN  NaN   
BBT    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   NaN   NaN  NaN   
BEN    N